# A guided example (code from Section 3.3)

In [1]:
"""
Adapted from ./msynth/scripts/symbolic_simplification.py
"""
from miasm.analysis.binary import Container
from miasm.analysis.machine import Machine
from miasm.core.locationdb import LocationDB
from miasm.ir.symbexec import SymbolicExecutionEngine

def getRaxExpr(file_path, start_addr):
    # symbol table
    loc_db = LocationDB()

    # open the binary for analysis
    container = Container.from_stream(open(file_path, 'rb'), loc_db)

    # cpu abstraction
    machine = Machine(container.arch)

    # init disassemble engine
    mdis = machine.dis_engine(container.bin_stream, loc_db=loc_db)

    # initialize intermediate representation
    lifter = machine.lifter_model_call(mdis.loc_db)

    # disassemble the function at address
    asm_block = mdis.dis_block(start_addr)

    # lift to Miasm IR
    ira_cfg = lifter.new_ircfg()
    lifter.add_asmblock_to_ircfg(asm_block, ira_cfg)

    # init symbolic execution engine
    sb = SymbolicExecutionEngine(lifter)

    # symbolically execute basic block
    sb.run_block_at(ira_cfg, start_addr)
    
    # return the expression of rax (return value)
    return sb.eval_exprid(lifter.arch.regs.RAX)

In [2]:
file_path = "./monster"
addr = 0x1149

rax_exp = getRaxExpr(file_path, addr)
print(f"RAX value from symbolic execution:\n---\n{rax_exp}\n")

RAX value from symbolic execution:
---
(RDI + RSI + (RDI * 0x6 + RSI * 0xFFFFFFFFFFFFFFFA + (RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) * 0x2 + (RDI + RSI + (RDI & RSI) * 0xFFFFFFFFFFFFFFFE + -(RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF))) * 0xFFFFFFFFFFFFFFFE + ((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0xFFFFFFFFFFFFFFFC + RSI * 0xFFFFFFFFFFFFFFFC + (RDI & RSI) * 0x2 + (RDI + (RDI & RSI) + (-RSI + (RDI | RSI)) * 0xFFFFFFFFFFFFFFFE + -RSI + (RDI ^ RSI)) * 0x2 + (RDI * 0xFFFFFFFFFFFFFFFE + RSI * 0x2 + (((RDI | RSI) ^ 0xFFFFFFFFFFFFFFFF) * 0xFFFFFFFFFFFFFFFE + -RDI + -(RDI | RSI) + 0xFFFFFFFFFFFFFFFE) * 0xFFFFFFFFFFFFFFFE + (-RSI + (RDI | RSI)) * 0x2 + -(RDI & RSI) + -(RDI ^ RSI) + 0xFFFFFFFFFFFFFFFF) * 0xFFFFFFFFFFFFFFFE + (-(RDI & RSI) + -(RDI ^ RSI) + 0xFFFFFFFFFFFFFFFF) * 0xFFFFFFFFFFFFFFFE + 0xFFFFFFFFFFFFFFFC) * 0x2 + ((RSI & (RDI ^ 0xFFFFFFFFFFFFFFFF)) + RDI * 0xFFFFFFFFFFFFFFFE + RSI * 0xFFFFFFFFFFFFFFFE + (RDI & RSI) * 0x2 + (-(RDI & RSI) + -(RDI ^ RSI) + 0xFFFFFFFFFFFFFFFF) * 0xFFFFFFFFFFFFFFFE 

In [3]:
from z3 import *
from miasm.ir.translators.z3_ir import TranslatorZ3

translator = TranslatorZ3()
rax_exp_z3 = translator.from_expr(rax_exp)
rax_exp_z3_simp = simplify(rax_exp_z3)
print(f"RAX after z3 simplification:\n---\n{rax_exp_z3_simp}\n")

RAX after z3 simplification:
---
128 +
49*RDI +
73*RSI +
18446744073709551592*~(~RSI | RDI) +
18446744073709551496*~(~RDI | ~RSI) +
126*(RDI | RSI) +
18446744073709551570*(RDI ^ RSI) +
128*~(RDI | RSI) |
59 +
RDI +
147*RSI +
18446744073709551570*~(~RDI | ~RSI) +
18446744073709551489*~(~RSI | RDI) +
18446744073709551596*(RDI | RSI) +
80*~(RDI | RSI) +
160*~(~RDI | RSI) +
60*(RSI | ~RDI) +
18446744073709551535*(RDI | ~RSI)



In [4]:
from msynth import Simplifier

# Path for the serialized oracle tables for QSynth
oracle_path = "./msynth/oracle.pickle"

# Initialize QSynth-based simplifier
synthSimplifier = Simplifier(oracle_path)

rax_exp_synth = synthSimplifier.simplify(rax_exp)
rax_exp_synth_z3 = translator.from_expr(rax_exp_synth)

print(f"RAX after synthesis simplification:\n---\n{rax_exp_synth_z3}\n")

RAX after synthesis simplification:
---
RDI + RSI | RDI ^ RSI



In [5]:
import sys
sys.path.append("./MBA-Solver/tools")

from mba_string_operation import verify_mba_unsat, variable_list
from svector_simplify import SvectorSimplify
from truthtable_search_simplify import PMBASimplify

def just_simplify_lmba(datafile):
    svObj = SvectorSimplify()
    
    with open(datafile, "rt") as fr:
        for line in fr:
            if "#" not in line:
                cmbaExpre = line.strip()
                vnumber = len(variable_list(cmbaExpre))
                simExpre = svObj.standard_simplify(cmbaExpre, vnumber)
                print(simExpre)
    
just_simplify_lmba("./monster.txt")

1*(x^y)
1*~(x|~y)+1*(x&~y)+2*(x&y)


In [6]:
RDI, RSI = BitVecs("RDI RSI", 64)

l_or = RDI^RSI
r_or = ~(RDI|~RSI) + (RDI&~RSI)  + 2*(RDI&RSI)

rax_exp_z3_simp_transform = l_or | r_or

print(f"RAX after MBA-Solver simplification on z3 simplified expression:\n---\n{rax_exp_z3_simp_transform}\n")

print(f"RAX synthesized candidate expression (recall) :\n---\n{rax_exp_synth_z3}\n")

print("Prove that synthesized candidate and MBA-Solver simplified expression are equivalent:\n---")
prove(rax_exp_z3_simp_transform == rax_exp_synth_z3)

RAX after MBA-Solver simplification on z3 simplified expression:
---
RDI ^ RSI | ~(RDI | ~RSI) + (RDI & ~RSI) + 2*(RDI & RSI)

RAX synthesized candidate expression (recall) :
---
RDI + RSI | RDI ^ RSI

Prove that synthesized candidate and MBA-Solver simplified expression are equivalent:
---
proved
